In [1]:
#1. Import Libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM, GRU, Bidirectional, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, mean_absolute_error, r2_score, classification_report
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

In [2]:
file_path = r"d:\IARE\The Project\Review 3\Codes\ev_charging_patterns_augmented.csv"
df = pd.read_csv(file_path)

In [3]:
df.dropna(subset=["Energy Consumed (kWh)", "Charging Duration (hours)", "Charging Rate (kW)", "Temperature (°C)"], inplace=True)

In [4]:
categorical_cols = ["Vehicle Model", "Charging Station Location", "Time of Day", "Day of Week", "Charger Type", "User Type"]
label_encoders = {col: LabelEncoder().fit(df[col]) for col in categorical_cols}
for col in categorical_cols:
    df[col] = label_encoders[col].transform(df[col])

In [6]:
df["Charging Start Time"] = pd.to_datetime(df["Charging Start Time"], dayfirst=True, errors='coerce')
df = df.dropna(subset=["Charging Start Time"])
df["Charging Hour"] = df["Charging Start Time"].dt.hour
df.drop(columns=["User ID", "Charging Station ID", "Charging Start Time", "Charging End Time"], inplace=True)

In [7]:
target = "Energy Consumed (kWh)"
features = ["Battery Capacity (kWh)", "Charging Duration (hours)", "Charging Rate (kW)", 
            "Charging Hour", "Time of Day", "Day of Week", "Temperature (°C)"]


In [8]:
scaler = StandardScaler()
df[features] = scaler.fit_transform(df[features])

In [9]:
X_train, X_test, y_train, y_test = train_test_split(df[features], df[target], test_size=0.2, random_state=42)


In [10]:
X_train_seq = X_train.values.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_seq = X_test.values.reshape(X_test.shape[0], X_test.shape[1], 1)


In [11]:
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-4)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)


In [15]:
import time

def build_and_train_model(model_type, X_train, y_train, X_test, y_test):
    start_time = time.time()
    model = Sequential()
    if model_type == "ANN":
        model.add(Dense(128, activation="relu", input_shape=(X_train.shape[1],)))
        model.add(Dense(64, activation="relu"))
        model.add(Dense(32, activation="relu"))
        model.add(Dense(1))
    elif model_type == "RNN":
        model.add(SimpleRNN(128, activation="relu", return_sequences=True, input_shape=(X_train.shape[1], 1)))
        model.add(Dropout(0.3))
        model.add(SimpleRNN(64, activation="relu"))
        model.add(Dense(32, activation="relu"))
        model.add(Dense(1))
    elif model_type == "LSTM":
        model.add(Bidirectional(LSTM(128, activation="relu", return_sequences=True, input_shape=(X_train.shape[1], 1))))
        model.add(Dropout(0.3))
        model.add(Bidirectional(LSTM(64, activation="relu")))
        model.add(Dense(32, activation="relu"))
        model.add(Dense(1))
    elif model_type == "GRU":
        model.add(Bidirectional(GRU(128, activation="relu", return_sequences=True, input_shape=(X_train.shape[1], 1))))
        model.add(Dropout(0.3))
        model.add(Bidirectional(GRU(64, activation="relu")))
        model.add(Dense(32, activation="relu"))
        model.add(Dense(1))
    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss="mse")
    model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test),
              verbose=1, callbacks=[lr_scheduler, early_stopping])
    elapsed_time = time.time() - start_time
    print(f"{model_type} training time: {elapsed_time:.2f} seconds")
    return model.predict(X_test), elapsed_time

In [17]:
y_pred_ann, ann_time = build_and_train_model("ANN", X_train, y_train, X_test, y_test)
y_pred_rnn, rnn_time = build_and_train_model("RNN", X_train_seq, y_train, X_test_seq, y_test)
y_pred_lstm, lstm_time = build_and_train_model("LSTM", X_train_seq, y_train, X_test_seq, y_test)
y_pred_gru, gru_time = build_and_train_model("GRU", X_train_seq, y_train, X_test_seq, y_test)

Epoch 1/100


c:\Users\Samrat Dudgundi\Anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


110/110 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 2022.7698 - val_loss: 569.8619 - learning_rate: 0.0010
Epoch 2/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 525.9066 - val_loss: 550.3895 - learning_rate: 0.0010
Epoch 3/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 510.6829 - val_loss: 539.7965 - learning_rate: 0.0010
Epoch 4/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 503.8195 - val_loss: 534.6407 - learning_rate: 0.0010
Epoch 5/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 490.9187 - val_loss: 531.7031 - learning_rate: 0.0010
Epoch 6/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 479.1823 - val_loss: 528.5015 - learning_rate: 0.0010
Epoch 7/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 478.7801 - val_loss: 527.4429 - learning_rate: 0.0010
Epoch 8/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 481.5415 - val_loss: 524.5319 - learning_rate: 0.0010
Epoch 9/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 492.3130 - val_loss: 521.1464 

c:\Users\Samrat Dudgundi\Anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


110/110 ━━━━━━━━━━━━━━━━━━━━ 8s 20ms/step - loss: 1349.4703 - val_loss: 639.3033 - learning_rate: 0.0010
Epoch 2/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 542.7113 - val_loss: 514.0070 - learning_rate: 0.0010
Epoch 3/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 508.6925 - val_loss: 570.6891 - learning_rate: 0.0010
Epoch 4/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 509.7988 - val_loss: 505.2113 - learning_rate: 0.0010
Epoch 5/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 473.4536 - val_loss: 505.7448 - learning_rate: 0.0010
Epoch 6/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 497.6781 - val_loss: 553.3844 - learning_rate: 0.0010
Epoch 7/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 538.6805 - val_loss: 497.8375 - learning_rate: 0.0010
Epoch 8/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 482.7914 - val_loss: 553.8582 - learning_rate: 0.0010
Epoch 9/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 506.0306 - val_loss: 

c:\Users\Samrat Dudgundi\Anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


110/110 ━━━━━━━━━━━━━━━━━━━━ 15s 31ms/step - loss: 1432.0348 - val_loss: 513.7450 - learning_rate: 0.0010
Epoch 2/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss: 492.6299 - val_loss: 510.5517 - learning_rate: 0.0010
Epoch 3/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss: 514.9786 - val_loss: 571.8292 - learning_rate: 0.0010
Epoch 4/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 69s 628ms/step - loss: 495.1979 - val_loss: 541.9871 - learning_rate: 0.0010
Epoch 5/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss: 497.5701 - val_loss: 505.5042 - learning_rate: 0.0010
Epoch 6/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 491.6252 - val_loss: 517.0480 - learning_rate: 0.0010
Epoch 7/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - loss: 491.7972 - val_loss: 505.0636 - learning_rate: 0.0010
Epoch 8/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - loss: 503.3193 - val_loss: 516.4399 - learning_rate: 0.0010
Epoch 9/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss: 465.1320 - val_los

c:\Users\Samrat Dudgundi\Anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


110/110 ━━━━━━━━━━━━━━━━━━━━ 16s 29ms/step - loss: 1405.8346 - val_loss: 514.6389 - learning_rate: 0.0010
Epoch 2/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - loss: 501.8940 - val_loss: 508.5057 - learning_rate: 0.0010
Epoch 3/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - loss: 500.6786 - val_loss: 514.3491 - learning_rate: 0.0010
Epoch 4/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 483.2015 - val_loss: 506.6125 - learning_rate: 0.0010
Epoch 5/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - loss: 509.1072 - val_loss: 520.1957 - learning_rate: 0.0010
Epoch 6/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 493.7298 - val_loss: 504.3867 - learning_rate: 0.0010
Epoch 7/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - loss: 481.2671 - val_loss: 503.0828 - learning_rate: 0.0010
Epoch 8/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss: 475.0073 - val_loss: 502.0872 - learning_rate: 0.0010
Epoch 9/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 480.0764 - val_loss:

In [18]:
def average_model_performance(model_type, X_train, y_train, X_test, y_test, runs=3):
    rmses, mapes, maes, r2s = [], [], [], []
    for _ in range(runs):
        y_pred, _ = build_and_train_model(model_type, X_train, y_train, X_test, y_test)
        rmse, mape, mae, r2 = evaluate_model(y_test, y_pred, model_type)
        rmses.append(rmse)
        mapes.append(mape)
        maes.append(mae)
        r2s.append(r2)
    print(f"{model_type} average RMSE: {np.mean(rmses):.4f}, MAPE: {np.mean(mapes):.4f}, MAE: {np.mean(maes):.4f}, R²: {np.mean(r2s):.4f}")

In [19]:
def evaluate_model(y_true, y_pred, model_name):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mape = mean_absolute_percentage_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    print(f"{model_name} - RMSE: {rmse:.4f}, MAPE: {mape:.4f}, MAE: {mae:.4f}, R² Score: {r2:.4f}")
    return rmse, mape, mae, r2

results = {
    "ANN": evaluate_model(y_test, y_pred_ann, "ANN"),
    "RNN": evaluate_model(y_test, y_pred_rnn, "RNN"),
    "LSTM": evaluate_model(y_test, y_pred_lstm, "LSTM"),
    "GRU": evaluate_model(y_test, y_pred_gru, "GRU")
}


ANN - RMSE: 21.8165, MAPE: 1.1552, MAE: 18.4706, R² Score: 0.0574
RNN - RMSE: 20.5941, MAPE: 0.9170, MAE: 16.7315, R² Score: 0.1601
LSTM - RMSE: 22.3278, MAPE: 1.2460, MAE: 19.2622, R² Score: 0.0127
GRU - RMSE: 21.7453, MAPE: 1.1418, MAE: 18.3222, R² Score: 0.0635
